# Sound Magician

Thời lượng ước tính: **120** phút

## Giới thiệu lab
Nâng cao! Có thể bạn đã từng xem một bộ phim hoặc chương trình truyền hình mà các điệp viên siêu hạng sử dụng một số kỹ thuật khó tin để nâng cao hình ảnh pixelated hoặc khôi phục một số dữ liệu bị mất. Trong lab này, bạn sẽ thực hiện điều tương tự — lần này mục tiêu của bạn là sử dụng hồi quy tuyến tính để khôi phục hoặc 'điền vào' phần bị xóa hoàn toàn của một tệp âm thanh!

Để hoàn thành lab này, bạn sẽ sử dụng FSDD (Free-Spoken-Digits-Dataset), một tập dữ liệu âm thanh do Zohar Jackson tổng hợp lại khi anh ấy nhận thấy không có nhiều âm thanh được làm sạch (không có khoảng trống, độ dài tương đương, cùng bitrate, cùng tỷ lệ mẫu trên giây, ...) thư viện âm thanh sẵn sàng cho machine learning.

In [1]:
import numpy as np
import pandas as pd

from sklearn.utils.validation import check_random_state
import scipy.io.wavfile as wavfile

### Về Âm thanh

Mẫu là các quan sát. Mỗi tệp âm thanh sẽ là một mẫu duy nhất trong tập dữ liệu.

Tìm thêm thông tin về [Mẫu âm thanh tại đây] (https://en.wikipedia.org/wiki/Sampling_(signal_processing)).

Mỗi tệp .wav thực sự chỉ là một loạt các mẫu số, được lấy mẫu từ tín hiệu analog. Sampling (Lấy mẫu) là một kiểu rời rạc. Khi chúng ta đề cập đến các 'sample (mẫu)', tức là các quan sát. Khi đề cập đến 'audio sample', chúng ta muốn nói đến "feature (thuộc tính)" thực sự của tệp âm thanh.

Mục tiêu của lab này là sử dụng hồi quy tuyến tính, đa mục tiêu để tạo ra bằng phép ngoại suy phần bị thiếu của tệp âm thanh thử nghiệm.

Mỗi một thuộc tính audio_sample sẽ là đầu ra của một phương trình, là một hàm của phần được cung cấp của audio_samples:

    missing_samples = f(provided_samples)

Bạn có thể thử với lượng âm thanh bạn muốn cắt và để máy tính tạo ra bằng tham số Provided_Portion.

Hãy sử dụng cái này. Đây là lượng tệp âm thanh sẽ được cung cấp, tính bằng phần trăm. Phần trăm còn lại của tệp sẽ được tạo thông qua phép ngoại suy tuyến tính.

In [2]:
Provided_Portion = 0.25

## 1. Chuẩn bị dữ liệu

Bắt đầu bằng cách tạo một list Python thông thường là `zero`:

In [3]:
# Nhập code của bạn ở đây
zero = []

Lặp qua tập dữ liệu và tải lên tất cả 50 tệp `0_jackson*.wav` bằng phương thức `wavfile.read()`: https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.io.wavfile.read.html từ thư mục [data](https://drive.google.com/drive/folders/1UDAvuRfXzcOcpI84neQAmAeffIrJKKBi?usp=sharing).

Hãy cẩn thận! `.read()` trả về một tuple và bạn chỉ quan tâm đến dữ liệu âm thanh chứ không phải sample_rate tại thời điểm này. Bên trong vòng lặp for, chỉ cần nối dữ liệu âm thanh đã tải vào list Python `zero`:

In [5]:
# Nhập code của bạn ở đây
import glob
list_audio = glob.glob("sound/data/*.wav")
for audio in list_audio:
    zero.append(wavfile.read(audio))

Hãy dành một chút thời gian để chuyển 0 thành một khung dữ liệu (DataFrame). Khi bạn làm như vậy, hãy đặt `dtype` thành `np.int16`, vì các tệp âm thanh đầu vào là 16 bit cho mỗi mẫu. Nếu bạn không biết cách thực hiện việc này, hãy đọc tài liệu tại đây: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html

Tiếc là những đoạn âm thanh này không được chuẩn hóa độ dài nên chúng ta sẽ phải chia nhỏ chúng để tất cả có cùng độ dài. Vì Pandas sẽ chèn các NAN tại bất kỳ vị trí nào để tạo thành mảng [n_observed_samples, n_audio_samples] hình chữ nhật hoàn hảo nên hãy thực hiện một `dropna` trên trục Y ở đây. Sau đó, chuyển đổi 1 trở lại thành NDArray bằng `yourarrayname.values`:

In [9]:
# Nhập code của bạn ở đây
df = pd.DataFrame(zero, dtype=np.int16)
df

C:\Users\ngoct\AppData\Local\Temp\ipykernel_1292\4210365783.py:2: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(zero, dtype=np.int16)


,0,1
0,8000,"[-369, -431, -475, -543, -571, -557, -528, -45..."
1,8000,"[-311, -91, -140, -182, -271, -68, -235, -359,..."
2,8000,"[-314, -303, -332, -376, -344, -344, -334, -29..."
3,8000,"[347, 351, 462, 451, 499, 509, 469, 534, 362, ..."
4,8000,"[-336, 160, 65, -161, -23, 61, 39, -332, 348, ..."
5,8000,"[354, 442, 610, 728, 768, 842, 873, 784, 739, ..."
6,8000,"[397, 531, 638, 699, 744, 769, 732, 694, 601, ..."
7,8000,"[382, 459, 530, 591, 641, 672, 690, 630, 592, ..."
8,8000,"[-393, 54, -71, -370, -75, 39, -234, -308, 79,..."
9,8000,"[-311, -363, -318, -476, -474, -500, -452, -39..."


Điều quan trọng là phải biết  (bao nhiêu mẫu audio_samples) độ dài dữ liệu hiện có.

`zero` hiện có shape giống như` [n_samples, n_audio_samples] `, vì vậy hãy lấy số lượng` n_audio_samples` và lưu trữ nó trong biến `n_audio_samples`:

In [ ]:
# Nhập code của bạn ở đây

## 2. Huấn luyện mô hình

Tạo mô hình hồi quy tuyến tính của bạn tại đây và lưu trữ nó trong biến `model`. Đừng vội huấn luyện hoặc làm bất cứ điều gì khác với nó:

In [ ]:
# Nhập code của bạn ở đây

Có 50 bản ghi mỗi clip. Bạn chỉ muốn lấy ngẫu nhiên một trong số chúng ra và cái đó sẽ KHÔNG được sử dụng trong quá trình huấn luyện mô hình. Nói cách khác, tệp mà chúng ta sẽ kiểm tra/chấm điểm sẽ là một mẫu không nhìn thấy, độc lập với phần còn lại của training set:

In [ ]:
# Hãy để nguyên dòng này cho đến khi bạn đã submit bài Lab của mình:
rng = check_random_state(7)

random_idx = rng.randint(zero.shape[0])
test  = zero[random_idx]
train = np.delete(zero, [random_idx], axis=0)

In ra shape của `train` và `test`.

`train` sẽ có dạng:`[n_samples, n_audio_samples]`, trong đó `n_audio_samples` là 'features' (thuộc tính) của tệp âm thanh

`test` sẽ có dạng `[n_audio_features]`, vì nó là mẫu duy nhất (tệp âm thanh, ví dụ: quan sát).

In [ ]:
# Nhập code của bạn ở đây

Dữ liệu thử nghiệm sẽ có hai phần, `X_test` và` y_test`.

`X_test` là phần đầu tiên của tệp âm thanh thử nghiệm mà chúng ta sẽ cung cấp cho máy tính làm input.

`y_test`,"nhãn" sẽ là phần còn lại của tệp âm thanh. Như vậy, máy tính sẽ sử dụng hồi quy tuyến tính để lấy ra phần bị thiếu của tệp âm thanh dựa trên dữ liệu huấn luyện mà nó nhận được!

Hãy lưu clip `test` ban đầu, clip mà bạn sắp xóa một nửa vào thư mục hiện tại để bạn có thể so sánh nó với clip 'đã vá' sau khi tạo. Bạn hẳn đã có `sample_rate` khi tải lên các tệp .wav: 

In [ ]:
wavfile.write('Original Test Clip.wav', sample_rate, test)

Chuẩn bị dữ liệu TEST bằng cách tạo lát `X_test`. Nó phải có các thuộc tính mẫu âm thanh `Provided_Portion` * `n_audio_samples`, được lấy từ tệp âm thanh thử nghiệm, hiện được lưu trữ trong biến `test`. Nói cách khác, lấy các thuộc tính âm thanh `Provided_Portion` * `n_audio_samples` ĐẦU TIÊN từ `test` và lưu trữ nó trong` X_test`. Điều này sẽ được thực hiện bằng cách sử dụng lập chỉ mục:

In [ ]:
# Nhập code của bạn ở đây

Nếu các thuộc tính `Provided_Portion` * `n_audio_samples` đầu tiên được lưu trữ trong `X_test`, thì chúng ta cũng cần lấy các thuộc tính âm thanh _còn lại_ và lưu trữ chúng trong` y_test`. Với các thuộc tính còn lại được lưu trữ trong đó, chúng ta sẽ có thể R ^ 2 "chấm điểm" thuật toán đã làm tốt như thế nào trong việc hoàn thành tệp âm thanh.

In [ ]:
# Nhập code của bạn ở đây

Lặp lại quy trình tương tự cho `X_train`, `y_train`. Sự khác biệt duy nhất là:

1. Bạn sẽ nhận được dữ liệu âm thanh từ `train` thay vì từ `test`
2. Bạn có nhớ shape của `train` mà bạn đã in ra trước đó không? Bạn muốn thực hiện việc cắt này nhưng đối với TẤT CẢ các mẫu (quan sát). Đối với mỗ quan sát, bạn muốn chia các thuộc tính âm thanh `Provided_Portion` * `n_audio_samples` đầu tiên thành `X_train`, và phần còn lại chuyển vào `y_train`. Có thể thực hiện tất cả những điều này bằng lập chỉ mục thông thường ở 2 dòng code:

In [ ]:
# Nhập code của bạn ở đây

SciKit-Learn sẽ 'tức giận' nếu bạn không cung cấp dữ liệu huấn luyện dưới dạng khung dữ liệu 2D có shape `[n_samples, n_features]`.

Vì vậy, nếu bạn chỉ có một MẪU, chẳng hạn như trường hợp của chúng ta với `X_test` và `y_test` thì bằng cách gọi `.reshape(1, -1)`, bạn có thể biến `[n_features]` thành `[1, n_features] `để đáp ứng SciKit-Learn.

Mặt khác, nếu bạn chỉ có một THUỘC TÍNH, bạn có thể gọi `.reshape (-1, 1)` trên dữ liệu của mình để biến `[n_samples]` thành `[n_samples, 1]`.

Định hình lại X_test và y_test thành [1, n_features]:

In [ ]:
# Nhập code của bạn ở đây

Điều chỉnh mô hình của bạn bằng cách sử dụng dữ liệu huấn luyện và nhãn:

In [ ]:
# Nhập code của bạn ở đây

## 3. Đánh giá mô hình

Sử dụng mô hình để dự đoán `label` của `X_test`. Lưu trữ dự đoán kết quả trong biến `y_test_prediction`:

In [ ]:
# Nhập code của bạn ở đây

SciKit-Learn sẽ sử dụng float64 để tạo các dự đoán của bạn, vì vậy hãy đưa các giá trị đó trở lại int16, đó là những gì tệp .wav cần:

In [ ]:
y_test_prediction = y_test_prediction.astype(dtype=np.int16)

Chấm điểm dự đoán của bạn sẽ hoạt động tốt như thế nào bằng cách truyền dữ liệu và nhãn thử nghiệm `y_test`:

In [ ]:
# Nhập code của bạn ở đây

In [ ]:
print("Extrapolation R^2 Score: ", score)

Hãy lấy phần `Provided_Portion` đầu tiên của clip thử nghiệm, phần mà bạn đã đưa vào mô hình hồi quy tuyến tính. Sau đó, kết hợp điều đó với thứ mà mô hình dự đoán đã tạo cho bạn và sau đó lưu đoạn âm thanh đã hoàn thành:

In [ ]:
completed_clip = np.hstack((X_test, y_test_prediction))
wavfile.write('Extrapolated Clip.wav', sample_rate, completed_clip[0])